<a href="https://colab.research.google.com/github/UdayRathod5277/ShadowFox/blob/main/LoanApproval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

loan_file_path = "/content/loan_prediction.csv"
loan_df = pd.read_csv(loan_file_path)

In [5]:
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 67.2+ KB


In [6]:
loan_df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
loan_df.drop(columns=["Loan_ID"], inplace=True)
#Useless Data

In [8]:
# Fill missing values
imputer = SimpleImputer(strategy="most_frequent")
loan_df.iloc[:, :] = imputer.fit_transform(loan_df)

In [9]:
encoder = LabelEncoder()
for col in ["Gender", "Married", "Education", "Self_Employed", "Property_Area", "Loan_Status"]:
    loan_df[col] = encoder.fit_transform(loan_df[col])

In [10]:
loan_df["Dependents"] = loan_df["Dependents"].replace({"3+": 3}).astype(int)

In [11]:
X = loan_df.drop(columns=["Loan_Status"])
y = loan_df["Loan_Status"]

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
y_pred = model.predict(X_test)

In [15]:
X_train.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
83,1,1,0,0,0,6000,2250.0,265.0,360.0,1.0,1


In [16]:
import numpy as np

new_applicant = {
    "Gender": "Male",
    "Married": "Yes",
    "Dependents": "1",
    "Education": "Graduate",
    "Self_Employed": "No",
    "ApplicantIncome": 5000,
    "CoapplicantIncome": 2000,
    "LoanAmount": 150,
    "Loan_Amount_Term": 360,
    "Credit_History": 1.0,
    "Property_Area": "Urban"
}

In [17]:
new_applicant_df = pd.DataFrame([new_applicant])
new_applicant_df = new_applicant_df[X.columns]

category_mappings = {
    "Gender": {"Male": 1, "Female": 0},
    "Married": {"Yes": 1, "No": 0},
    "Education": {"Graduate": 1, "Not Graduate": 0},
    "Self_Employed": {"Yes": 1, "No": 0},
    "Property_Area": {"Urban": 2, "Semiurban": 1, "Rural": 0}
}

In [18]:
for col, mapping in category_mappings.items():
    if col in new_applicant_df.columns:
        new_applicant_df[col] = new_applicant_df[col].map(mapping)

In [19]:
if "Dependents" in new_applicant_df.columns:
    new_applicant_df["Dependents"] = new_applicant_df["Dependents"].replace({"3+": 3}).astype(int)

In [20]:
new_applicant_df = new_applicant_df.reindex(columns=loan_df.columns, fill_value=0)
new_applicant_df = pd.DataFrame(imputer.transform(new_applicant_df), columns=loan_df.columns)

In [21]:
new_applicant_df = new_applicant_df.drop(columns=['Loan_ID', 'Loan_Status'], errors='ignore')

In [22]:
new_applicant_df = new_applicant_df[X_train.columns]

In [23]:
prediction = model.predict(new_applicant_df)

In [24]:
if prediction[0] == 1:
    print("Loan Status: Loan Approved")
else:
    print("Loan Status: Loan Not Approved")

Loan Status: Loan Approved
